In [2]:
import requests
import os
from datetime import date
 
from requests.exceptions import HTTPError 
 

In [3]:
endpoint="https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_transactions.json"
req = requests.get(endpoint, allow_redirects=True)

In [ ]:
# req.text

In [5]:
import ast
dictionary = ast.literal_eval(req.text)

In [16]:
import pandas as pd

df = pd.DataFrame(dictionary)

In [25]:
df.amount

0       $15,001 - $50,000
1       $15,001 - $50,000
2       $15,001 - $50,000
3        $1,001 - $15,000
4        $1,001 - $15,000
              ...        
7349              Unknown
7350              Unknown
7351              Unknown
7352              Unknown
7353              Unknown
Name: amount, Length: 7354, dtype: object

In [27]:
df = df[df.amount != "Unknown"]

In [29]:
df = df[df.ticker != "--"]

In [30]:
len(df)

6575

In [34]:
df.amount[0].split(' - ')[1]

'$50,000'

In [35]:
df.amountMin = df.amount.split(' - ')[0]

AttributeError: 'Series' object has no attribute 'split'

In [ ]:
# print(dictionary)

In [ ]:
for i , j in enumerate(dictionary):
    if dictionary[i]['amount'] == 'Unknown':
        print(i, dictionary[i])
        del dictionary[i]

In [ ]:
for i , j in enumerate(dictionary):
    if dictionary[i]['ticker'] == '--':
        print(i, dictionary[i])
        del dictionary[i]

In [12]:
len(dictionary)

7354

In [ ]:
# for i , j in enumerate(dictionary):
#     print(dictionary[i]['amount'])

In [ ]:
# for i , j in enumerate(dictionary):
#     print(dictionary[i]['amountMin'])

In [14]:
dictionary[2134]['amount']

'Unknown'

In [ ]:
for i , j in enumerate(dictionary):
    print(i,dictionary[i]['amount'].split(' - ')[1])

In [32]:
for i , j in enumerate(dictionary):
    dictionary[i]['amountMax'] = dictionary[i]['amount'].split(' - ')[1]

IndexError: list index out of range

In [5]:
# Table creation script

import sqlite3

try:
    sqliteConnection = sqlite3.connect('../db.sqlite3')
    sqlite_create_table_query = '''CREATE TABLE StockTable (
                                id INTEGER PRIMARY KEY,
                                ticker TEXT,
                                asset_description TEXT,
                                asset_type TEXT,
                                amount TEXT,
                                type TEXT,
                                transaction_date DATETIME,
                                senator TEXT);'''

    cursor = sqliteConnection.cursor()
    print("Successfully Connected to SQLite")
    cursor.execute(sqlite_create_table_query)
    sqliteConnection.commit()
    print("SQLite table created")

    cursor.close()

except sqlite3.Error as error:
    print("Error while creating a sqlite table", error)
finally:
    if sqliteConnection:
        sqliteConnection.close()
        print("sqlite connection is closed")

Successfully Connected to SQLite
Error while creating a sqlite table table StockTable already exists
sqlite connection is closed


In [33]:
#Table add record script
import sqlite3

def add_record(i):
    try:
        sqliteConnection = sqlite3.connect('../db.sqlite3')
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")

        sqlite_insert_query = """INSERT INTO StockTable
                            (ticker, asset_description, asset_type, amount, type, transaction_date, senator) 
                            VALUES 
                            (?,?,?,?,?,?,?)"""
        data_tuple = (dictionary[i]["ticker"],
                        dictionary[i]["asset_description"],
                        dictionary[i]["asset_type"],
                        dictionary[i]["amount"],
                        dictionary[i]["type"],
                        dictionary[i]["transaction_date"],
                        dictionary[i]["senator"])
        count = cursor.execute(sqlite_insert_query, data_tuple)
        sqliteConnection.commit()
        print("Record inserted successfully into SqliteDb_developers table ", cursor.rowcount)
        cursor.close()
    except sqlite3.Error as error:
        print("Failed to insert data into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")

In [58]:
dictionary[8759]

{'transaction_date': '07/25/2012',
 'disclosure_date': '07/25/2012',
 'owner': 'N/A',
 'ticker': 'N/A',
 'asset_description': 'This filing was disclosed via scanned PDF. Use link in ptr_link column to view the PDF.',
 'asset_type': 'PDF Disclosed Filing',
 'type': 'N/A',
 'amount': 'Unknown',
 'comment': '',
 'senator': 'Benjamin L Cardin',
 'ptr_link': 'https://efdsearch.senate.gov/search/view/paper/CDFDAF62-18EA-4298-B0C5-62085A6EC3CD/'}

In [ ]:
for i,j in enumerate(dictionary):
    add_record(i)

In [2]:
# Script to delete all records from table
import sqlite3
def deleteRecord():
    try:
        sqliteConnection = sqlite3.connect('../db.sqlite3')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        # Deleting single record now
        sql_delete_query = """DELETE from StockTable"""
        cursor.execute(sql_delete_query)
        sqliteConnection.commit()
        print("Record deleted successfully ")
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to delete record from sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("the sqlite connection is closed")

deleteRecord()

Connected to SQLite
Failed to delete record from sqlite table no such table: StockTable
the sqlite connection is closed


In [40]:
len(dictionary)

8760

In [42]:
type(dictionary)

list

In [ ]:
import pprint
for i in range(385,400):
    pprint.pprint(dictionary[i])